# GF Summer School 2021

## Accelerated Text - low code natural language generation platform

TODO AT image

### Why low code?

TODO bullet list from https://medium.com/accelerated-text/no-code-a-perfect-fit-for-natural-language-generation-bbe3027d22ed?source=collection_home---6------1-----------------------

### Building blocks

* Visual programming language - Blockly
* Document Plans
* AMR
* Grammatical Framework (RGL)

### Usage example - document plan developer perspective

### Jupyter Notebook - data scientist (end user) perspective

### Build Document Plan from scratch

* Very simple sentence, showing how data connects to DP how AMR and RGLs are connected
* Show prebuilt more complex RGL grammar, and how it grows with AMR, DP
* Explain Semantic Graph and how it helps to automate some of the grammar programming

### Prototype based grammar creation

* Parse non trivial sentence
* Show how it can be abstracted. Going from a single example sentence to more possibilities.
* Go back to JN to emphasise this part - ease of use for the end user - close the presentation.


## Conclusions

TODO